In [1]:
import pandas as pd
import numpy as np
import scipy.optimize as sco
from numpy import linalg
# https://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.optimize.minimize.html

# COBYLA
# slower than SLSQP
# more stable with respect to obtaining a solution than SLSQP
# cannot use bounds
# only inequality can be used as a constraint, not equality
# https://org-technology.com/posts/scipy-constrained-minimization-of-multivariate-scalar-functions.html#
# https://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.optimize.minimize.html

# https://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.optimize.minimize.html
# https://quantu.hateblo.jp/entry/2017/12/27/010526

# Cov = pd.DataFrame(np.zeros([4,4]))
# Cov.loc[0,:]=[0.0449016,0.0396086,0.0442209,0.0323200]
# Cov.loc[1,:]=[0.0396086,0.0733868,0.0543290,0.0357016]
# Cov.loc[2,:]=[0.0442209,0.0543290,0.0689063,0.0400982]
# Cov.loc[3,:]=[0.0323200,0.0357016,0.0400982,0.0530842]

Cov = np.zeros((4,4))
Cov[0,:]=[0.0449016,0.0396086,0.0442209,0.0323200]
Cov[1,:]=[0.0396086,0.0733868,0.0543290,0.0357016]
Cov[2,:]=[0.0442209,0.0543290,0.0689063,0.0400982]
Cov[3,:]=[0.0323200,0.0357016,0.0400982,0.0530842]

In [2]:



def f(w):
    SD = ( w @ Cov @ w.T )**0.5
    return SD 

def opts(m,C):
    SD0 = np.diag(Cov**0.5)
    tmp = 1/SD0   
    w0 = tmp/sum(tmp)

    if m == "SLSQP":
        B= [(0, 1)] * len(Cov)
        opts = sco.minimize(fun=f, x0=w0, method=m, bounds=B, constraints=C)
    else:
        opts = sco.minimize(fun=f, x0=w0, method=m, constraints=C)

    ww = opts["x"]
    SD = ( ww @ Cov @ ww.T)**0.5
    RC = ww * ( ww @ Cov ) / SD

    tmp = ["Weight","SD","W*SD","RC","RC%"]
    Summary = pd.DataFrame(np.zeros([len(tmp),len(Cov)]),index=tmp)
    Summary.loc["Weight",:] = ww
    Summary.loc["SD",:] = SD0
    Summary.loc["W*SD",:] = ww * SD0
    Summary.loc["RC",:] = RC
    Summary.loc["RC%",:] = RC/SD

    for i in range(len(Summary)):
        Summary.loc[Summary.index[i],"Net"] = sum( Summary.loc[Summary.index[i],list(range(len(Cov)))] )
        Summary.iloc[i,:] = Summary.iloc[i,:].apply("{:.1%}".format)

    Summary.loc["SD","Net"] = "-"
    return Summary
    
    
m = "SLSQP"  #Method
C = [{"type": "eq", "fun": lambda w: sum(w) - 1}]
Summary = opts(m,C)

print("MinVariance/SLSQP")
Summary

MinVariance/SLSQP


,0,1,2,3,Net
Weight,58.0%,5.9%,0.0%,36.1%,100.0%
SD,21.2%,27.1%,26.3%,23.0%,-
W*SD,12.3%,1.6%,0.0%,8.3%,22.2%
RC,11.6%,1.2%,0.0%,7.2%,20.0%
RC%,58.0%,5.9%,0.0%,36.1%,100.0%


In [3]:
m = "COBYLA"  #Method
c0 = {"type": "ineq", "fun": lambda w: -sum(w) + 1-0.0001}
c1 = {"type": "ineq", "fun": lambda w: sum(w) - 1.000}
c2 = {"type": "ineq", "fun": lambda w:  min(w) - 0.0001 } #min(w) >= 0.001 
C = [c0, c1, c2 ]
Summary = opts(m,C)

print("MinVariance/COBYLA")
Summary

MinVariance/COBYLA


,0,1,2,3,Net
Weight,58.2%,5.5%,0.0%,36.3%,100.0%
SD,21.2%,27.1%,26.3%,23.0%,-
W*SD,12.3%,1.5%,0.0%,8.4%,22.2%
RC,11.6%,1.1%,0.0%,7.3%,20.0%
RC%,58.2%,5.5%,0.0%,36.3%,100.0%


In [4]:
d = [.012,.01,.015,.01] #Ret
SD_U = 0.15  #RiskLimit

def f(w):
    Ret = d @ w
    return -Ret

def opts(m,C):
    SD0 = np.diag(Cov**0.5)
    tmp = 1/SD0   
    w0 = tmp/sum(tmp)

    if m == "SLSQP":
        B= [(0, 1)] * len(Cov)
        opts = sco.minimize(fun=f, x0=w0, method=m, bounds=B, constraints=C)
    else:
        opts = sco.minimize(fun=f, x0=w0, method=m, constraints=C)

    ww = opts["x"]
    SD = ( ww @ Cov @ ww.T )**0.5
    RC = ww * ( ww @ Cov ) / SD

    tmp = ["Weight","Ret","SD","W*Ret","W*SD","RC","RC%"]
    Summary = pd.DataFrame(np.zeros([len(tmp),len(Cov)]),index=tmp)
    Summary.loc["Weight",:] = ww
    Summary.loc["Ret",:] = d
    Summary.loc["SD",:] = SD0
    Summary.loc["W*Ret",:] = ww * d
    Summary.loc["W*SD",:] = ww * SD0
    Summary.loc["RC",:] = RC
    Summary.loc["RC%",:] = RC/SD

    for i in range(len(Summary)):
        Summary.loc[Summary.index[i],"Net"] = sum( Summary.loc[Summary.index[i],list(range(len(Cov)))] )
        Summary.iloc[i,:] = Summary.iloc[i,:].apply("{:.1%}".format)

    Summary.loc[["Ret","SD"],"Net"] = "-"
    return Summary
    
m = "SLSQP"  #Method
c0 = {"type": "ineq", "fun": lambda w: -sum(w) + 1}
c1 = {"type":"ineq", "fun": lambda w: - ( w @ Cov @ w.T)**0.5 + SD_U}

C = [c0,c1]
Summary = opts(m,C)

print("MaxRet")
Summary

# m = "COBYLA"  #Method
# c0 = {"type": "ineq", "fun": lambda w: -sum(w) + 1}
# c1 = {"type": "ineq", "fun": lambda w:  min(w) - 0.0001 }
# c2 = {"type": "ineq", "fun": lambda w: -( w @ Cov @ w.T )**0.5 + SD_U}
# C = [c0,c1,c2]
# Summary = opts(m,C)
# print("MaxRet")
# Summary

MaxRet


,0,1,2,3,Net
Weight,34.1%,0.0%,30.7%,3.1%,67.9%
Ret,1.2%,1.0%,1.5%,1.0%,-
SD,21.2%,27.1%,26.3%,23.0%,-
W*Ret,0.4%,0.0%,0.5%,0.0%,0.9%
W*SD,7.2%,0.0%,8.1%,0.7%,16.0%
RC,6.8%,0.0%,7.7%,0.5%,15.0%
RC%,45.3%,0.0%,51.2%,3.5%,100.0%


In [5]:
#RiskParity
def f(w):
    SD = (w @ Cov @ w.T )**0.5
    RC = w * ( w @ Cov ) / SD
    return sum( (RC / SD - 1/len(w))**2 ) 

B= [(0.001, 1)] * len(Cov)
C = [{"type": "eq", "fun": lambda w: sum(w) - 1}]


SD0 = np.diag(Cov**0.5)
tmp = 1/SD0   
w0 = tmp/sum(tmp)

opts = sco.minimize(fun=f, x0=w0, method="SLSQP", bounds=B, constraints=C)

ww = opts["x"]
SD = ( ww @ Cov @ ww.T )**0.5
RC = ww * ( ww @ Cov ) / SD

tmp = ["Weight","SD","W*SD","RC","RC%"]
Summary = pd.DataFrame(np.zeros([len(tmp),len(Cov)]),index=tmp)
Summary.loc["Weight",:] = ww
Summary.loc["SD",:] = SD0
Summary.loc["W*SD",:] = ww * SD0
Summary.loc["RC",:] = RC
Summary.loc["RC%",:] = RC/SD

for i in range(len(Summary)):
    Summary.loc[Summary.index[i],"Net"] = sum( Summary.loc[Summary.index[i],list(range(0,len(Cov)))] )
    Summary.iloc[i,:] = Summary.iloc[i,:].apply("{:.1%}".format)

Summary.loc["SD","Net"] = "-"
print("RiskParity")
Summary

RiskParity


,0,1,2,3,Net
Weight,27.9%,22.6%,22.0%,27.6%,100.0%
SD,21.2%,27.1%,26.3%,23.0%,-
W*SD,5.9%,6.1%,5.8%,6.4%,24.1%
RC,5.3%,5.3%,5.3%,5.3%,21.1%
RC%,25.0%,25.0%,25.0%,25.0%,100.0%


In [6]:
#FactorParity
from scipy.linalg import null_space
A = pd.DataFrame(np.zeros([4,3]))
A.loc[0,:]=[0.9,0,0.5]
A.loc[1,:]=[1.1,0.5,0]
A.loc[2,:]=[1.2,0.3,0.2]
A.loc[3,:]=[0.8,0.1,0.7]
pinvA = np.linalg.pinv(A)
pinvB = np.linalg.pinv(A.T)
pinvB_NS = null_space(pinvB.T)
pinvB_NS *= np.sign(pinvB_NS[0,0])
B_Bar = np.c_[pinvB, pinvB_NS]
B_Tilde = np.linalg.inv(B_Bar)[len(Cov)-1,:]


def f(w):
    SD = ( w @ Cov @ w.T)**0.5
    RC_F = (A.T @ w) * (pinvA @ Cov @ w) / SD
    RC_Tilde_F = (B_Tilde @ w ) * ( B_Tilde @ Cov @ w) / SD
    return sum( (RC_F / SD - 1/len(RC_F))**2 ) 

B = [(0.001, 1)] * len(Cov)
C = [{'type': 'eq', 'fun': lambda w: sum(w) - 1}]
w0 =[1. / len(Cov)] * len(Cov)

opts = sco.minimize(fun=f, x0=w0, method='SLSQP', bounds=B, constraints=C)

SD0 = np.diag(Cov**0.5)
ww = opts["x"]
SD = ( ww @  Cov @ ww.T )**0.5
RC = ww * ( ww @ Cov ) / SD

tmp = ["Weight","SD","W*SD","RC","RC%"]
Summary = pd.DataFrame(np.zeros([len(tmp),len(Cov)]),index=tmp)
Summary.loc["Weight",:] = ww
Summary.loc["SD",:] = SD0
Summary.loc["W*SD",:] = ww * SD0
Summary.loc["RC",:] = RC
Summary.loc["RC%",:] = RC/SD

for i in range(len(Summary)):
    Summary.loc[Summary.index[i],"Net"] = sum( Summary.loc[Summary.index[i],list(range(0,len(Cov)))] )
    Summary.iloc[i,:] = Summary.iloc[i,:].apply("{:.1%}".format)

Summary.loc["SD","Net"] = "-"

RC_F = ( A.T @ ww ) * ( pinvA @ Cov @ ww) / SD
RC_Tilde_F = ( B_Tilde @ ww ) * ( B_Tilde @ Cov @ ww) / SD
sum(RC_F,RC_Tilde_F)


print("Port byAsset")
Summary

Port byAsset


,0,1,2,3,Net
Weight,0.1%,39.3%,0.5%,60.1%,100.0%
SD,21.2%,27.1%,26.3%,23.0%,-
W*SD,0.0%,10.6%,0.1%,13.9%,24.6%
RC,0.0%,9.1%,0.1%,12.7%,21.9%
RC%,0.1%,41.5%,0.4%,58.0%,100.0%


In [7]:
tmp = ["RC"]
Summary2 = pd.DataFrame(np.zeros([len(tmp),len(Cov)]), index=tmp, columns=list(range(0,len(Cov)-1)) + ["Additional"])
Summary2.loc["RC",:] = list(RC_F) + [RC_Tilde_F]
Summary2.loc["RC","Net"] = sum(Summary2.loc["RC",:])

Summary2.iloc[0,:] = Summary2.iloc[0,:].apply("{:.2%}".format)
print("SD_byFactor:")
Summary2

SD_byFactor:


,0,1,2,Additional,Net
RC,7.28%,7.28%,7.28%,0.04%,21.89%
